In [1]:
import pymongo
import re
import pprint
from pymongo import MongoClient
import ete3
from ete3 import NCBITaxa
import pandas as pd
ncbi = NCBITaxa()
client = MongoClient()
db = client.COCONUT2020
collection = db.uniqueNaturalProduct

In [2]:
data = collection.find({'coconut_id': 'CNP0228556'}, {'_id': 0, 'coconut_id': 1, 'textTaxa': 1, 'taxid': 1})
tables = pd.DataFrame(list(data))

In [3]:
# Method to put upper and lower text_taxas in the database
text_taxa_upper = {}
a = 0
nx = ['notax']
length = len(tables.index)
while a < length:
    actual = tables.loc[a,'textTaxa']
    if actual == nx:
        text_taxa_upper[a] = actual
        a += 1
    else:
        lista = []
        b = 0
        long = len(actual)
        while b < long:
            actual1 = actual[b].lower()
            palabras = actual1.split(maxsplit = 3)
            palabras[0] = palabras[0].lower().capitalize()
            actual2 = ' '.join(palabras)
            lista.append(actual2) 
            b += 1
        text_taxa_upper[a] = lista
        a += 1

In [4]:
# Method to fix some names -- it will be better
fixes = ['Actinoplanes "sp. (ma 7066; atcc 55532)"'] # for this, I suggest you to first query NCBI taxonomy with the whole name. If this doesn't work, to remove everything after "sp." (using regex or simple split on "sp.")
fixes1 = ['Plants', 'Plant']
fixes2 = ['Animal', 'Animals']
# fixes2 = ['spp.', 'sp.'] future changes
for key in text_taxa_upper:
    w = 0
    x = text_taxa_upper[key]
    # palabras_upper = x.split(maxsplit = 4)
    lg = len(x)
    while w < lg:
        if x[w] in fixes:
            x[w] = 'Actinoplanes sp.'
            w += 1
        elif x[w] in fixes1:
            x[w] = 'Viridiplantae'
            w += 1
        elif x[w] in fixes2:
            x[w] = 'Metazoa'
            w +=1 
        elif x[w] == 'Unknown-fungus sp. mf 5638, a nalanthamala sp':
            x[w] = 'Nalanthamala'
            w += 1
        elif x[w] == 'Unknown-fungus sterila' or 'Unknown-fungus sterilia' in x[w]:
            x[w] = 'Sterila'
            w += 1
        elif x[w] == 'Unknown-fungus bcc 54265 (botryosphaeriaceae)':
            x[w] = 'Botryosphaeriaceae'
            w += 1
        elif x[w] == 'Unknown-fungus dothideales (fungus) mpuc 046':
            x[w] = 'Dothideales'
            w += 1
        elif 'Unknown-fungus' in x[w]:
            x[w] = 'Fungi'
            w += 1
        elif 'Unknown-bacterium' in x[w]:
            x[w] = 'Bacteria'
            w += 1
        else:
            x[w] = x[w]
            w += 1
        text_taxa_upper[key] = x

In [5]:
# Method to create a column in tables with the text_taxas in upper and lower
names = []
for key in text_taxa_upper:
    x = text_taxa_upper[key]
    names.append(x)
# Text_taxa_curated son los nombres con las mayúsculas y minúsculas bien
tables['text_taxa_curated'] = names

In [7]:
# Method to obtain the id's names of the taxid from database
# Método para obtener los nombres de los ids de taxid
ids_to_names = {}
n = 0
while n < length:
    current_id = tables.loc[n,'taxid']
    # Boolean of a empty list is False
    if bool(current_id):
        diccionary = {}
        list_names = []
        long = len(current_id)
        m = 0
        # Nombre de todos los id ordenados y metidos en un diccionario. Finalemente se quedará con un diccionario de listas
        while m < long:
            current_id1 = [current_id[m]]
            name = ncbi.get_taxid_translator(current_id1)
            if name == {}:
                diccionary[m] = ['notax']
                m += 1
            else:
                diccionary[m] = name
                m += 1
        for key in diccionary:
            x = diccionary[key]
            list_names.append(x)
        ids_to_names[n] = list_names
        n += 1
    else:
        ids_to_names[n] = ['notax']
        n += 1
    
# Method to create in tables a column with the translated id of the database
# Método para escribir en tables los nombres traducidos y los id que había en taxid
names_id_bbdd = []
for key in ids_to_names:
    x = ids_to_names[key]
    names_id_bbdd.append(x)
# id_to_text_taxa son los nombres traducidos de los id que ya hay en la bbdd junto con sus ids
tables['id_to_text_taxa'] = names_id_bbdd

In [8]:
# Method to obtain a list of names without the ids from de database
# Método para obtener una lista de nombres sin sus ids de los que hay en la bbdd
# Lista de nombres que eran de los id del id_to_text_taxa
names_bbdd = {}
k = 0
while k < length:
    current_idname = tables.loc[k,'id_to_text_taxa']
    if current_idname == nx:
        names_bbdd[k] = current_idname
        k += 1
    else:
        list_idname = []
        long = len(current_idname)
        l = 0
        while l < long:
            current_idname2 = current_idname[l]
            if current_idname2 == nx:
                list_idname.append(current_idname2)
            else:
                for key in current_idname2:
                    key1 = int(key)
                    valor = current_idname2.get(key)
                    list_idname.append(valor)
            l += 1
        names_bbdd[k] = list_idname
        k += 1
        
# Create a new column in tables with the only the names translated from taxids
only_names_id_bbdd = []
for key in names_bbdd:
    x = names_bbdd[key]
    only_names_id_bbdd.append(x)
tables['names_id_text_taxa'] = only_names_id_bbdd

In [9]:
# Method to translate the curated_text_taxas to ids (too slow)
textTaxas_to_ids = {}
mr = ['Marine', '2-f"', '2-3"']
for index, row in tables.iterrows():
    current_text = row['text_taxa_curated']
    if current_text == nx:
        textTaxas_to_ids[index] = [0]
    else:
        dicc = {}
        list_trans_ids = []
        lon = len(current_text)
        e = 0
        while e < lon:
            current_text2 = [current_text[e]]
            if current_text[e] in mr:
                dicc[e] = [0]
                e += 1
            else:
                trans_id = ncbi.get_name_translator(current_text2)
                if trans_id == {}:
                    dicc[e] = [0]
                else:
                    dicc[e] = trans_id
                e += 1
        for key in dicc:
            x = dicc[key]
            list_trans_ids.append(x)
        textTaxas_to_ids[index] = list_trans_ids
# textTaxas_to_ids contains {}, {}, {}, {'Brucea javanica': [210348]} ... exists empty ones but I change for id 0

In [10]:
# Add a column in tables with the text_taxas_curated and the ids translated
names_curated_to_ids = []
for key in textTaxas_to_ids:
    x = textTaxas_to_ids[key]
    names_curated_to_ids.append(x)
tables['names_curated_to_ids'] = names_curated_to_ids

In [11]:
#Method to obtain a list of id from the names_curated_to_ids (only ids)
ids_bbdd_curated = {}
s = 0
while s < length:
    current_nameid = tables.loc[s,'names_curated_to_ids']
    if current_nameid == [0]:
        ids_bbdd_curated[s] = current_nameid
        s += 1
    elif current_nameid == []:
        ids_bbdd_curated[s] = [0]
        s += 1
    else:
        list_ids_namecurated = []
        lh = len(current_nameid)
        z = 0
        while z < lh:
            current_nameid2 = current_nameid[z]
            if current_nameid2 == [0]:
                list_ids_namecurated.append(current_nameid2)
            else:
                for key in current_nameid2:
                    numero = current_nameid2.get(key)
                    list_ids_namecurated.append(numero)
            z += 1
        ids_bbdd_curated[s] = list_ids_namecurated
        s += 1
        
# Create a column in tables only with the ids from the names in tax names curated
only_ids_names_curated_bbdd = []
for i in ids_bbdd_curated:
    lista_paso_medio = []
    x = ids_bbdd_curated[i]
    if x == [0]:
        only_ids_names_curated_bbdd.append(x)
    elif len(x) == 1:
        only_ids_names_curated_bbdd.append(x[0])
    else:
        lo = len(x)
        g = 0
        lista_medio = []
        while g < lo:
            y = x[g]
            lista_medio.append(y[0])
            g += 1
        only_ids_names_curated_bbdd.append(lista_medio)
tables['ids_names_curated'] = only_ids_names_curated_bbdd

In [12]:
# Get a list of the ids and ranks to build the database

ranks_ids = []
list_ids = []
for index, row in tables.iterrows():
    current_ids_list = row['taxid']
    current_names_list = row['ids_names_curated']
    if current_ids_list != []:
        for num in current_ids_list:
            list_ids.append(num)
            rank = ncbi.get_rank([num])
            if rank == {}:
                ranks_ids.append({num: 'no_rank'})
            else:
                ranks_ids.append(rank)
    
    if current_names_list != [0]:
        for num in current_names_list:
            if num != 0:
                list_ids.append(num)
                rank = ncbi.get_rank([num])
                if rank == {}:
                    ranks_ids.append({num: 'no_rank'})
                else:
                    ranks_ids.append(rank)

ranks_dicc = {}
for x in ranks_ids:
    current_rank = list(x.values())
    for key in x:
        ranks_dicc[key] = current_rank[0]

In [15]:
tables.to_json("only1.json")

In [16]:
print(ranks_dicc)

{3641: 'species', 83213: 'species', 4442: 'species', 9606: 'species', 10090: 'species', 197426: 'species', 123438: 'species', 82457: 'species', 190243: 'species', 120635: 'species', 1202421: 'species', 425106: 'species', 152363: 'species', 138017: 'species', 1127084: 'species', 422353: 'species', 361442: 'species', 1822162: 'species', 103484: 'species', 155230: 'species', 1115353: 'species', 153547: 'species', 452773: 'species', 115664: 'species', 89194: 'species', 1489822: 'species', 354523: 'species', 146505: 'species', 73976: 'species', 4182: 'species', 159066: 'species', 76963: 'species', 715731: 'species', '1764327': 'no_rank', 185544: 'species', 1040932: 'species', 535968: 'species', 1475404: 'species', 353880: 'species', 414126: 'species', 137226: 'species', 459127: 'species', 50880: 'species', 1745164: 'species', 213301: 'varietas', 37656: 'species', 49390: 'species', 339904: 'species', 319927: 'species', 185542: 'species', 13443: 'species', 49373: 'species', 2708: 'species', 3

In [22]:
rank = ['superkingdom', 'species', 'no_rank', 'phylum', 'genus', 'kingdom', 'subspecies', 'varietas', 'family', 'subfamily', 'strain', 'suborder', 'subgenus', 'forma specialis', 'clade', 'forma', 'biotype', 'order' ]
rank_list = []
count_dicc = {}
for key in ranks_dicc:
    x = ranks_dicc[key]
    rank_list.append(x)

for i in rank:
    count = rank_list.count(i)
    count_dicc[i] = count

In [23]:
print(count_dicc)

{'superkingdom': 0, 'species': 64, 'no_rank': 1, 'phylum': 0, 'genus': 0, 'kingdom': 2, 'subspecies': 0, 'varietas': 3, 'family': 0, 'subfamily': 0, 'strain': 0, 'suborder': 0, 'subgenus': 0, 'forma specialis': 0, 'clade': 0, 'forma': 0, 'biotype': 0, 'order': 0}


In [13]:
# Create the new database
db2 = client.COCONUT2020
col = db2.curatedNP
for index, row in tables.iterrows():
    myid = row['coconut_id']
    list_ids = row['taxid']
    list_names = row['text_taxa_curated']
    list_onlynames_id = row['names_id_text_taxa']
    list_onlyid_names = row['ids_names_curated']
    total = len(list_names)
    list_data = {'_id': myid}
    list_data['tax_classification'] = []
    
    # First with the names
    if list_onlyid_names != [0]: # I don't know if add list_names != nx to be complete sure // while or if??
        t = 0
        while t < total:
            if list_onlyid_names[t] == 0:
                t += 1
            else:
                try:
                    rank_id = ncbi.get_rank([list_onlyid_names[t]])
                    if rank_id == {}:
                        list_data['tax_classification'].append({'taxonomy': [{'name': list_names[t], 'taxid': list_onlyid_names[t], 'rank': 'no_rank'}]})
                    else:
                        for key in rank_id:
                            rank_name = rank_id[key]
                        list_data['tax_classification'].append({'taxonomy': [{'name': list_names[t], 'taxid': list_onlyid_names[t], 'rank': rank_name}]})
                    t += 1
                except:
                    list_data['tax_classification'].append({'taxonomy': [{'name': list_names[t], 'taxid': list_onlyid_names[t], 'rank': 'no_rank'}]})
                    t += 1
        
    # Second with the ids #Comprobar
    total2 = len(list_ids)
    if list_onlynames_id != nx:
        c = 0
        while c < total2:
            if list_onlynames_id[c] == nx:
                c += 1
            else:
                try:
                    rank_id = ncbi.get_rank([list_ids[c]])
                    if rank_id == {}:
                        list_data['tax_classification'].append({'taxonomy': [{'name': list_onlynames_id[c], 'taxid': list_ids[c], 'rank': 'no_rank'}]})
                    else:
                        for key in rank_id:
                            rank_name = rank_id[key]
                        list_data['tax_classification'].append({'taxonomy': [{'name': list_onlynames_id[c], 'taxid': list_ids[c], 'rank': rank_name}]})
                    c += 1
                except:
                    list_data['tax_classification'].append({'taxonomy': [{'name': list_onlynames_id[c], 'taxid': list_ids[c], 'rank': 'no_rank'}]})
                    c += 1
        
    if list_data['tax_classification'] != []:
        col.insert_one(list_data)

DuplicateKeyError: E11000 duplicate key error collection: COCONUT2020.curatedNP index: _id_ dup key: { _id: "CNP0228556" }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 'CNP0228556'}, 'errmsg': 'E11000 duplicate key error collection: COCONUT2020.curatedNP index: _id_ dup key: { _id: "CNP0228556" }'}

In [28]:
rank = ['superkingdom', 'species', 'no_rank', 'no rank', 'phylum', 'genus', 'kingdom', 'subspecies', 'varietas', 'family', 'subfamily', 'strain', 'suborder', 'subgenus', 'forma specialis', 'clade', 'forma', 'biotype', 'order' ]
cuenta = col.count_documents({'_id': 'CNP0228556','rank': 'species'})
print(str(cuenta))

0


In [29]:
col.find_one({'_id': 'CNP0228556'}, {'tax_classification.taxonomy.rank': 1, '_id': 0})

{'tax_classification': [{'taxonomy': [{'rank': 'varietas'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'kingdom'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  {'taxonomy': [{'rank': 'species'}]},
  